Sarthak Dandotiya **17BCE0203**

## Assesment 2

### Question 1
Write a program to show the implementation of breadth-first and depth-first crawler.

In [5]:
import urllib.request
import re
from bs4 import BeautifulSoup 
import pandas as pd

class Crawler():

    def __init__(self, url, branching_factor):
        self.url = url
        self.branching_factor = branching_factor
        self.DFS = 'dfs'
        self.BFS = 'bfs'

    def get_url(self, current_url, urls_found):
        urls = set([])
        try:
            response = urllib.request.urlopen(current_url)
        except:
            print('Error')
            return urls
        soup = BeautifulSoup(response.read())
        newLink = soup.findAll('a')
        for link in newLink:
            try:
                link = str(link.get('href'))
                m = re.match(r'http+', link)
                if not m:
                    link = current_url + link
                if link not in urls_found:
                    urls.add(str(link))
            except UnicodeEncodeError as e:
                print ('Error ---> ', e)
        return urls

    def start_dfs(self):
        return self.start(self.DFS)

    def start_bfs(self):
        return self.start(self.BFS)

    def start(self, algorithm):
        structure = []
        urls_found = set([])
        structure.append(self.url)
        count = 0
        while count < self.branching_factor and structure:
            if algorithm == self.BFS:
                current_url = structure.pop(0)
            else:
                current_url = structure.pop()
            urls = self.get_url(current_url, urls_found)
            print(('[%s] links found in %s') %(len(urls), current_url))
            for i in urls:
                structure.append(i)
                urls_found.add(i)
            count += 1
        print("\n\n","---",len(urls_found),"---")
        return urls_found

    def __str__(self):
        return ('url:%s branching_factor:%s') %(self.url, self.branching_factor)

crawl = Crawler('https://en.wikipedia.org/wiki/Cambridge_Analytica',10)
print("\n\n--------------DFS------------------\n\n")
dfs = crawl.start_dfs()
print("\n\n--------------BFS------------------\n\n")
bfs = crawl.start_bfs()





--------------DFS------------------


[887] links found in https://en.wikipedia.org/wiki/Cambridge_Analytica
[701] links found in https://en.wikipedia.org/wiki/Cambridge_Analytica#cite_note-102
Error
[0] links found in https://en.wikipedia.org/wiki/Cambridge_Analytica#cite_note-102#cite_note-69
[701] links found in https://en.wikipedia.org/wiki/Cambridge_Analytica#cite_note-102/wiki/Cambridge_University
[701] links found in https://en.wikipedia.org/wiki/Cambridge_Analytica#cite_note-102/wiki/Cambridge_University/wiki/UK_Independence_Party
[701] links found in https://en.wikipedia.org/wiki/Cambridge_Analytica#cite_note-102/wiki/Cambridge_University/wiki/UK_Independence_Party/w/index.php?title=Cambridge_Analytica&action=edit&section=15
Error
[0] links found in https://en.wikipedia.org/wiki/Cambridge_Analytica#cite_note-102/wiki/Cambridge_University/wiki/UK_Independence_Party/w/index.php?title=Cambridge_Analytica&action=edit&section=15#cite_note-Rosenburg_M-36
[701] links found in https

### Question 2
Write a program to crawl two different websites, extract the content and save them in
separate text documents. Based on the contents of those two documents, prepare
one inverted index.

In [1]:
from __future__ import unicode_literals

import nltk
from nltk.corpus import stopwords
import requests
from bs4 import BeautifulSoup
from os import system, name
import os
import re
import pandas as pd
import numpy as np


def save(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content)
    for script in soup(["script", "style"]):
        script.extract()
    text = soup.get_text()
    cleanString1 = re.sub('\W+',' ', text )
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split(" "))
    modifiied_chunks = remove_stopwords(chunks)
    text = '\n'.join(chunk for chunk in modifiied_chunks if chunk)
    return text

def remove_stopwords(words):
    my_stopwords = my_stopwords = set(stopwords.words('english'))
    my_stopwords.update([".", ",", "'", ":", ";", "\"", "...",\
                         "(", ")", "{", "}", "[", "]", "©", "@",\
                         "'m", "--", "__", "?", "|", "#", "!",\
                         "-", "$", "'s", "&", "·", "–", "’", "‘",\
                         "“", "”", "%", "''", "+", "/","-",])

    filtered1 = [w for w in words if not w in my_stopwords]
    return filtered1

def cleaning_data(words):
    removed_links = re.sub('^\( & \)&', ' ',words)
    return removed_links

def process_files(filenames):
    file_to_terms = {}
    for file in filenames:
        pattern = re.compile('[\W_]+')
        file_to_terms[file] = open(file, 'r',encoding='utf-8').read().lower();
        file_to_terms[file] = pattern.sub(' ',file_to_terms[file])
        re.sub(r'[\W_]+','', file_to_terms[file])
        file_to_terms[file] = file_to_terms[file].split()
    return file_to_terms


def index_one_file(termlist):
    fileIndex = {}
    for index, word in enumerate(termlist):
        if word in fileIndex.keys():
            fileIndex[word].append(index)
        else:
            fileIndex[word] = [index]
    return fileIndex

def make_indices(termlists):
    total = {}
    for filename in termlists.keys():
        total[filename] = index_one_file(termlists[filename])
    return total


def fullIndex(regdex):
    total_index = {}
    for filename in regdex.keys():
        for word in regdex[filename].keys():
            
            if word in total_index.keys():
                if filename in total_index[word].keys():
                    total_index[word][filename].extend(regdex[filename][word][:])
                else:
                    total_index[word][filename] = regdex[filename][word]
            else:
                total_index[word] = {filename: regdex[filename][word]}
    return total_index

def one_word_query(word, invertedIndex):
    pattern = re.compile('[\W_]+')
    word = pattern.sub(' ',word)
    if word in invertedIndex.keys():
        return [filename for filename in invertedIndex[word].values()]
    else:
        return []
    
    
def free_text_query(string,index):
    pattern = re.compile('[\W_]+')
    string = pattern.sub(' ',string)
    result = []
    for word in string.split():
        result += one_word_query(word,index)
    return list(set(result))


def phrase_query(string, invertedIndex):
    pattern = re.compile('[\W_]+')
    string = pattern.sub(' ',string)
    listOfLists, result = [],[]
    for word in string.split():
        listOfLists.append(free_text_query(word,invertedIndex))
    setted = set(listOfLists[0]).intersection(*listOfLists)
    for filename in setted:
        temp = []
        for word in string.split():
            temp.append(invertedIndex[word][filename][:])
        for i in range(len(temp)):
            for ind in range(len(temp[i])):
                temp[i][ind] -= i
        if set(temp[0]).intersection(*temp):
            result.append(filename)
        print('\n temp : \n')
        print(temp)
    return result

urls = ["https://en.wikipedia.org/wiki/Web_mining", "https://en.wikipedia.org/wiki/Machine_learning"]

for url in urls:
    text = save(url)
    with open(str(url.split("/")[-1])+'.txt','w', encoding='utf-8') as file:
        file.write(text)

filenames = ["WM.txt", "ML.txt"]

# filenames = []
# for i in os.listdir():
#     if i.endswith('.txt'):
#         filenames.append(i)

termslist = process_files(filenames)

totaldict = make_indices(termslist)

index = fullIndex(totaldict)

inv_matrix = pd.DataFrame(index)
inv_matrix = inv_matrix.T
inv_matrix.rename(columns = {"Machine_learning.txt":"Document1", "Web_mining.txt":"Document2"},inplace=True)

def check_nan(val):
    if isinstance(val, float):
        return 0
    elif isinstance(val, list):
        return ('<' + str(len(val)) + ' ' +  str(val) + '>') 
    
inv_matrix = inv_matrix.applymap(check_nan)
display(inv_matrix.T)

inv_matrix.T.to_csv(r'inv.csv')

,web,mining,is,the,application,of,data,techniques,to,discover,...,analysis,through,unsupervised,its,across,business,problems,referred,predictive,analytics
ML.txt,0,<1 [137]>,"<6 [3, 32, 90, 106, 138, 163]>","<5 [4, 68, 102, 119, 131]>","<2 [128, 157]>","<6 [7, 37, 79, 121, 133, 141]>","<4 [50, 54, 136, 150]>",0,"<7 [16, 57, 66, 94, 109, 130, 166]>",0,...,<1 [151]>,<1 [152]>,<1 [153]>,<1 [156]>,<1 [158]>,<1 [159]>,<1 [160]>,<1 [165]>,<1 [168]>,<1 [169]>
WM.txt,"<17 [0, 16, 28, 43, 74, 84, 87, 90, 102, 126, ...","<10 [1, 7, 26, 76, 92, 104, 128, 150, 167, 200]>","<3 [2, 22, 77]>","<17 [3, 13, 18, 27, 71, 83, 96, 108, 112, 115,...",<1 [4]>,"<10 [5, 32, 73, 98, 111, 123, 159, 165, 176, 1...","<2 [6, 39]>","<2 [8, 205]>","<8 [9, 35, 49, 51, 78, 106, 171, 190]>","<2 [10, 107]>",...,0,0,0,0,0,0,0,0,0,0


### Question 3
Write a program to find out rank of each documents prepared in question number 2
with respect to a Query using TF-IDF. (Show the rank of each document based on
both Cosine similarity and Euclidean Distance.)

In [8]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
import pandas as pd


def stopw(f):
    stoplist = stopwords.words('english')
    file = open(f,'r',encoding='utf-8')
    text = file.read()
    stop_words = stopwords.words('english')
    stop_words=set(stop_words)
    stop_words.update([".", ",", "'", ":", ";", "\"", "...",\
                         "(", ")", "{", "}", "[", "]", "©", "@",\
                         "'m", "--", "__", "?", "|", "#", "!",\
                         "-", "$", "'s", "&", "·", "–", "’", "‘",\
                         "“", "”", "%", "''", "+", "/","-","``"])
    word_tokens = word_tokenize(text)
    num_of_lines = sum(1 for line in file)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    filtered_sentence = []
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w.lower())
    file.close
    return filtered_sentence

filtered_sentencea= stopw("ML.txt")
filtered_sentenceb= stopw('WM.txt')

wordSet = set(filtered_sentencea).union(set(filtered_sentenceb))

wordDictA = dict.fromkeys(wordSet, 0)
wordDictB = dict.fromkeys(wordSet, 0)


for word in filtered_sentencea:
    wordDictA[word]+= 1
    
for word in filtered_sentenceb:
    wordDictB[word]+=1

    
def computeTF(wordDict, bow):
    tfDict = {}
    bowCount = len(bow)
    for word, count in wordDict.items():
        tfDict[word] = count/float(bowCount)
    return tfDict

tfA = computeTF(wordDictA, filtered_sentencea)
tfB = computeTF(wordDictB, filtered_sentenceb)


def computeIDF(docList):
    import math
    idfDict = {}
    N = len(docList)

    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for doc in docList:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1

    for word, val in idfDict.items():
        idfDict[word] = math.log(N / float(val))
        
    return idfDict

idfs = computeIDF([wordDictA, wordDictB])


def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return tfidf

tfidfA = computeTFIDF(tfA, idfs)
tfidfB = computeTFIDF(tfB, idfs)

df = pd.DataFrame([tfidfA, tfidfB],)
df.head()

# print("TFIDF:")

df.rename(index={0:'Doc1',1:'Doc2'}, inplace=True)
# display(df.head())


def process_query(query):
    query_arr = query.split(" ")
    qur_arr = {word:query_arr.count(word)/len(query_arr) for word in query_arr}
    modified_arr = {}
    for i in df.columns:
        if i in list(qur_arr.keys()):
            modified_arr[i] = qur_arr[i]
        else:
            modified_arr[i] = 0
  
    return modified_arr
    
def calculate_euclidean_Dist(v1,v2):
    v1 = v1.astype('float64')
    mult_resul = v1.sub(v2,fill_value=0)**2
    return sum(list(mult_resul))**0.5
    
def calculate_cosine(v1,v2):
    v1 = v1.astype('float64')
    sum_res = v1.mul(v2,fill_value=0)
    return sum(list(sum_res))

query = "web mining in machine learning"
# query = "web mining"
# query = "machine learning"
tf_query = process_query(query)
tf_idf_query = computeTFIDF(tf_query,idfs)
# print("For Query- ",tf_idf_query)
df['EUCLIDIAN'] = 0
df['COSINE'] = 0
for i in range(len(df.index)):
    df.iloc[i,-2] = calculate_euclidean_Dist(df.loc[list(df.index)[i],:], pd.Series(tf_idf_query))
    df.iloc[i,-1] = calculate_cosine(df.loc[list(df.index)[i],:], pd.Series(tf_idf_query))

print("TFIDF:")
display(df.head())

def rank_docs(cosine_sr,euc_sr):
    cosin_sr_list = cosine_sr.sort_values(ascending=False)
    euc_sr_list = euc_sr.sort_values()
    for i in range(len(euc_sr_list)):
        if list(cosin_sr_list.index)[i] == list(cosin_sr_list.index)[i]:
            print(str(i+1) + ' -> '+ list(cosin_sr_list.index)[i])

print("Document Ranking:")
rank_docs(df.iloc[:,-1],df.iloc[:,-2])

TFIDF:


,accessing,across,activities,algorithm,algorithms,also,analysis,analytics,another,apparatuses,...,vision,web,website,wide,within,without,world,would,EUCLIDIAN,COSINE
Doc1,0.000000,0.00608,0.000000,0.00608,0.018241,0.0,0.00608,0.00608,0.000000,0.000000,...,0.00608,0.000000,0.000000,0.0,0.00608,0.01216,0.000000,0.000000,0.241437,0.013486
Doc2,0.005059,0.00000,0.005059,0.00000,0.000000,0.0,0.00000,0.00000,0.005059,0.005059,...,0.00000,0.086011,0.005059,0.0,0.00000,0.00000,0.005059,0.005059,0.258699,0.011924


Document Ranking:
1 -> Doc1
2 -> Doc2
